In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-thesis7/RAW_StudeeDB_AppFormData_Thesis7 (1).csv
/kaggle/input/thesis-datasets/RAW_StudeeDB_Nationalities_Thesis2.csv
/kaggle/input/thesis-datasets/RAW_StudeeDB_AdmissionsCriteria_Thesis6.csv
/kaggle/input/thesis-datasets/RAW_StudeeDB_StudentClicks_Thesis3.csv
/kaggle/input/thesis-datasets/RAW_StudeeDB_UniversityData_Thesis5.csv
/kaggle/input/thesis-datasets/RAW_StudeeDB_ApplicationData_Thesis1.csv
/kaggle/input/applicationdata-application-student-and-program-id/Result_4.csv
/kaggle/input/full-program-list/PROD_Full_Program_List_Published.csv


In [3]:
!pip install scikit-surprise

In [4]:
import numpy as np
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder  # Added OneHotEncoder import
import os
from sklearn.metrics.pairwise import cosine_similarity

# Load the datasets
df_application = pd.read_csv('/kaggle/input/thesis-datasets/RAW_StudeeDB_ApplicationData_Thesis1.csv')
df_clicks = pd.read_csv('/kaggle/input/thesis-datasets/RAW_StudeeDB_StudentClicks_Thesis3.csv')
df_university = pd.read_csv('/kaggle/input/thesis-datasets/RAW_StudeeDB_UniversityData_Thesis5.csv')
df_full_program_list = pd.read_csv('/kaggle/input/full-program-list/PROD_Full_Program_List_Published.csv')
df_app_student_program_uuid = pd.read_csv('/kaggle/input/applicationdata-application-student-and-program-id/Result_4.csv')

In [5]:
print("Application Data")
print(df_application.head())

Application Data
  application_account_url       BIN_TO_UUID(`application_uuid`)  \
0                     NaN  233554bf-f8bd-4992-942a-981d8d10c9a3   
1                     NaN  3c0d1548-7d33-46c1-9014-127489d54eea   
2                     NaN  007c863c-3b79-44b3-8577-8aef521a623c   
3                     NaN  7d42783a-2233-45c1-b71c-7f20532717ad   
4                     NaN  21aa66c2-2ba4-45e6-9ce4-1220737e8f7c   

     created_date_time   BIN_TO_UUID(`education_agent_uuid`)  \
0  2022-07-30 07:48:34                                   NaN   
1  2023-08-25 11:29:15  34003144-3725-4846-8578-bd7af0373910   
2  2023-10-09 08:40:55                                   NaN   
3  2024-07-21 06:14:49                                   NaN   
4  2022-09-02 10:16:50                                   NaN   

                         email  first_name  \
0     Merdimakiadi10@gmail.com       Merdi   
1        proffababio@yahoo.com      EGBUKA   
2  gideonrockson1999@gmail.com      Gideon   
3   mothibi

In [6]:
print("Student Clicks Data")
print(df_clicks.head())

Student Clicks Data
  BIN_TO_UUID(`city_uuid`)             BIN_TO_UUID(`click_uuid`)  \
0                      NaN  000050ed-8f99-4035-b0d7-574560ee6dd8   
1                      NaN  00037b21-71b5-43b6-bb3e-68208919ed0d   
2                      NaN  0003f992-3ed1-4042-8ad0-ff2c6b436451   
3                      NaN  000493bd-ea6a-4821-b9b4-797102eaa3c6   
4                      NaN  00052ed0-f59e-47b5-b484-1844c4fadb67   

  BIN_TO_UUID(`country_uuid`)    created_date_time isas_processed_date_time  \
0                         NaN  2022-07-22 07:11:09                      NaN   
1                         NaN  2022-02-06 21:55:19      2022-02-11 06:39:20   
2                         NaN  2021-01-06 01:18:29      2021-05-12 11:29:21   
3                         NaN  2021-03-08 05:48:03      2021-05-12 11:29:21   
4                         NaN  2022-09-19 08:54:28                      NaN   

      BIN_TO_UUID(`program_level_uuid`)           BIN_TO_UUID(`program_uuid`)  \
0              

In [7]:
print("University Data")
print(df_university.head())

University Data
               BIN_TO_UUID(`city_uuid`)          BIN_TO_UUID(`currency_uuid`)  \
0  b6ee89dc-cea9-4bed-9436-93843b79b050  37ac0c29-7f04-4adc-be67-cdf087371ee5   
1  528da28a-51dc-4444-eeb0-debae73e3fba  023e8e35-7d2a-40f0-befd-9022186ad8cc   
2  69865792-405d-4930-20a5-e8ff078d3d91  0dc1b638-c4d0-4dfb-b675-246e35e677d1   
3  46ecdca2-a396-4d75-81d5-5b135d31f92f  023e8e35-7d2a-40f0-befd-9022186ad8cc   
4  04fc689f-6cf5-4284-84c7-06a1c09eec1f  0dc1b638-c4d0-4dfb-b675-246e35e677d1   

  BIN_TO_UUID(`default_university_contact_uuid`)  is_commercial  \
0           670c3cb4-e363-4700-963e-9e82d228ed24           True   
1           574b5304-7240-4a78-a443-c54b96179090           True   
2           5ee34e02-4f65-4a7a-8cb4-8d023986cdb1           True   
3           20ecb5d5-f9fa-4cec-951b-1b989df23fcf           True   
4                                            NaN           True   

  is_deposit_required  university_acceptance_rate  university_allows_agents  \
0             U

In [8]:
print(df_full_program_list.head())

   program_tuition_fees_per_year           BIN_TO_UUID(`program_uuid`)  \
0                          28718  d65f94a9-666c-4a8a-a326-875798f7f346   
1                          23013  b93882e7-490e-4116-812e-d8c3db88d77c   
2                          23428  d497f56a-7700-4464-910b-728d9a1b92be   
3                          29880  a6e49bd4-bfa4-447c-a98e-bcab1a084297   
4                          16900  5881c3ac-f93a-44af-8f84-02134e632ca4   

       state        BIN_TO_UUID(`university_uuid`)  \
0  published  72a8fa1e-9da4-4a60-b5e2-3cd2d2839146   
1  published  a16d7569-0957-4053-afed-d29f5c1d67b5   
2  published  8bb1e3e5-b150-451d-bf9f-45225e5a0aa7   
3  published  c53b0001-cc11-4d48-846f-550b9686dfbf   
4  published  4e8f84d6-db0d-4503-8734-5b4fb92c463c   

                                        program_name  \
0                       Rehabilitation Sciences - BS   
1                               Computer Engineering   
2  Dietetics and Nutritional Sciences, Pre-Profes...   
3  Rob

In [9]:
print(df_app_student_program_uuid.head())

                       Student_App_UUID                          Student_UUID  \
0  0000eb21-655d-4bd6-8641-e7f5898e35a3  ffd9efba-0b60-496e-b16f-f39fbb7c7832   
1  00026801-704a-4518-87b6-1ed5a501f4d8  f51da60a-0b64-4c5e-991c-9f951ce54566   
2  0007d663-f56d-423d-abe3-ebe7e3c4f702  0f63919b-fa66-46f3-a67d-68ba22c181d3   
3  0008ab69-3760-4767-8ef6-b7dedda479b4  3b0dcaa6-99cc-4c2e-a153-b5b2f786fb32   
4  0008c4a3-dbd0-4ae7-bf67-59716e4ab077  cc4e91e7-4e00-444e-9d95-f3a186e1aff6   

                           Program_UUID                   Program_Intake_UUID  
0  6f082f72-51be-4944-baf4-41b4f1a81202  5db6e13f-8d81-11ec-b755-02cd10dd1184  
1  f7fb376d-9749-46be-80f8-14bdfca24e6b  603e2492-8d81-11ec-b755-02cd10dd1184  
2  7a91acab-67d7-4c5a-bf68-53a0e8200ebc  aba5f8f6-abc6-4a45-9793-1f85a4956616  
3  331545b5-45b9-4591-9db0-5e0ec109a4be  a9e89c9e-c6c9-4f04-acd8-fafbd0e1a42d  
4  c136aedf-d866-4beb-9bbf-a85f0b00e499  5f40db61-8d81-11ec-b755-02cd10dd1184  


In [10]:
# Merge Application data and Clicks data on `student_uuid`
df_merged = pd.merge(df_application, df_clicks, how='inner', on='BIN_TO_UUID(`student_uuid`)')

# Merge the resulting dataset with University data on `university_uuid`
df_merged = pd.merge(df_merged, df_university, how='inner', on='BIN_TO_UUID(`university_uuid`)')

# Merge the merged dataset with the full program list to add program details
df_merged = pd.merge(df_merged, df_full_program_list, how='left', on='BIN_TO_UUID(`program_uuid`)')


# Check the merged dataset
df_merged.head()


In [ ]:
# Add click count per student and program combination
df_click_counts = df_merged.groupby(['BIN_TO_UUID(`student_uuid`)', 'BIN_TO_UUID(`program_uuid`)']).size().reset_index(name='click_count')
# Merge the click count data back into the main DataFrame
df_cleaned_with_clicks = pd.merge(df_merged, df_click_counts, on=['BIN_TO_UUID(`student_uuid`)', 'BIN_TO_UUID(`program_uuid`)'], how='left')
# Display the cleaned data
print(df_cleaned_with_clicks.head())

In [ ]:
# Select important columns

# Now select important columns, including 'BIN_TO_UUID(`student_application_uuid`)'
df_cleaned = df_cleaned_with_clicks[[
    'BIN_TO_UUID(`student_uuid`)',  # Ensure column name is correct
    'BIN_TO_UUID(`university_uuid`)_y',  # Ensure column name is correct
    'BIN_TO_UUID(`student_application_uuid`)',  # Include this column
    'student_click_source', 
    'BIN_TO_UUID(`program_uuid`)',  # Ensure column name is correct
    'university_rank', 
    'university_acceptance_rate',
    'BIN_TO_UUID(`program_level_uuid`)',  # Ensure column name is correct
    'click_count',
    'program_name',  # From full program list
    'slug',  # From full program list
    'program_tuition_fees_per_year',  # Added tuition fees from full program list
    'university_type',  # University info
    'university_location_address',  # University location
    'created_date_time_x'  # Interaction timestamp
]]

# Rename columns for easier reference
df_cleaned = df_cleaned.rename(columns={
    'BIN_TO_UUID(`student_uuid`)': 'student_uuid',
    'BIN_TO_UUID(`program_uuid`)': 'program_uuid',
    'BIN_TO_UUID(`university_uuid`)_y': 'university_uuid',
    'BIN_TO_UUID(`student_application_uuid`)': 'student_application_uuid',  # Renaming the new column
    'program_tuition_fees_per_year': 'tuition_fee',
    'created_date_time_x': 'created_date_time',
    'BIN_TO_UUID(`program_level_uuid`)': 'program_level_uuid'
})

# Display cleaned data
print(df_cleaned.head())


In [ ]:
# Handle missing values in 'click_count', 'program_name', and 'tuition_fee'
df_cleaned['click_count'] = df_cleaned['click_count'].fillna(1)  # Replace NaN with 1 for missing click counts
df_cleaned['program_name'] = df_cleaned['program_name'].fillna('Unknown Program')  # Handle missing program names
df_cleaned['tuition_fee'] = df_cleaned['tuition_fee'].fillna(df_cleaned['tuition_fee'].mean())  # Fill missing tuition fees with the average

In [ ]:
# Cap tuition fees at the 95th percentile
cap_value = df_cleaned['tuition_fee'].quantile(0.95)
df_cleaned['tuition_fee'] = np.where(df_cleaned['tuition_fee'] > cap_value, cap_value, df_cleaned['tuition_fee'])

# Cap university rank at the 95th percentile
cap_value_rank = df_cleaned['university_rank'].quantile(0.95)
df_cleaned['university_rank'] = np.where(df_cleaned['university_rank'] > cap_value_rank, cap_value_rank, df_cleaned['university_rank'])

# Optionally, categorize the university ranks into tiers
df_cleaned['university_rank_category'] = pd.cut(
    df_cleaned['university_rank'], 
    bins=[0, 1000, 5000, 10000, 20000, np.inf],
    labels=['Top 1000', 'Top 5000', 'Top 10000', 'Top 20000', 'Others']
)

In [ ]:
# Define some common subjects and keywords
subject_keywords = {
    'Business': ['business', 'management', 'administration', 'finance', 'accounting'],
    'Engineering': ['engineering', 'mechanical', 'civil', 'electrical', 'software'],
    'Marketing': ['marketing', 'advertising', 'digital'],
    'Health': ['health', 'medicine', 'nursing', 'pharmacy', 'biomedical'],
    'Arts': ['arts', 'design', 'creative', 'literature'],
    'Law': ['law', 'legal'],
    'Science': ['science', 'biology', 'chemistry', 'physics'],
    'Technology': ['technology', 'computer', 'information', 'software', 'data'],
}

# Function to assign subjects based on program name
def assign_subject(program_name):
    for subject, keywords in subject_keywords.items():
        if any(keyword.lower() in program_name.lower() for keyword in keywords):
            return subject
    return 'Other'

In [ ]:
# Apply the function to the dataset
df_cleaned['subject'] = df_cleaned['program_name'].apply(assign_subject)

# Normalize tuition fee and university rank
scaler = MinMaxScaler()
df_cleaned['normalized_tuition_fee'] = scaler.fit_transform(df_cleaned[['tuition_fee']])
df_cleaned['normalized_university_rank'] = scaler.fit_transform(df_cleaned[['university_rank']])

In [ ]:
# Handle NaN values by replacing with 0 or column mean before applying cosine_similarity
df_cleaned['normalized_university_rank'] = df_cleaned['normalized_university_rank'].fillna(0)
df_cleaned['normalized_tuition_fee'] = df_cleaned['normalized_tuition_fee'].fillna(0)

In [ ]:
# Step 3: Calculate Similarity for Content-based Filtering
subject_encoder = OneHotEncoder()
subject_encoded = subject_encoder.fit_transform(df_cleaned[['subject']]).toarray()

In [ ]:
# Adjust the weights for different features (subjects, tuition fees, rank, etc.)
weight_subject = 0.6
weight_tuition = 0.2
weight_rank = 0.2

In [ ]:
# Combine the weighted content features
content_features = np.hstack((
    weight_subject * subject_encoded,
    weight_tuition * df_cleaned[['normalized_tuition_fee']].values,
    weight_rank * df_cleaned[['normalized_university_rank']].values
))

In [ ]:
# Combine features for similarity calculation
content_features = np.hstack((subject_encoded,df_cleaned[['normalized_tuition_fee', 'normalized_university_rank']].values))

# Calculate similarity matrix
content_similarity_matrix = cosine_similarity(content_features)

# Store similarity in a DataFrame
content_similarity_df = pd.DataFrame(content_similarity_matrix, index=df_cleaned['program_uuid'], columns=df_cleaned['program_uuid'])

In [ ]:
# Step 4: Collaborative Filtering using SVD
# Set the rating scale based on the max click count from filtered data
reader = Reader(rating_scale=(1, df_cleaned['click_count'].max()))
data = Dataset.load_from_df(df_cleaned[['student_uuid', 'program_uuid', 'click_count']], reader)

In [ ]:
# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2)

In [ ]:
# Train the model using SVD
svd_model = SVD(n_factors=400, n_epochs=200, lr_all=0.005, reg_all=0.02)
svd_model.fit(trainset)

In [ ]:
# Function to get the top subject(s) the student interacted with
def get_student_top_subjects(student_id):
    # Get the subjects of programs the student has interacted with
    student_programs = df_cleaned[df_cleaned['student_uuid'] == student_id]
    subjects = student_programs['subject'].value_counts()
    
    # Return the top subject(s)
    return subjects.index.tolist()

In [ ]:
# Hybrid recommendation function with similarity score threshold
def get_hybrid_recommendations(student_id, top_n=5):
    # Step 1: Get top SVD-based recommendations for the student
    svd_predictions = svd_model.test([(student_id, program_id, 0) for program_id in df_cleaned_filtered['program_uuid']])

    # Step 2: Sort the predictions based on the SVD estimated rating (descending order)
    svd_predictions_sorted = sorted(svd_predictions, key=lambda x: x.est, reverse=True)

    # Step 3: Collect top SVD-based programs (ensure uniqueness)
    svd_top_programs = list(dict.fromkeys([pred.iid for pred in svd_predictions_sorted]))

    # Get the student's top subject(s) from past interactions
    student_top_subjects = get_student_top_subjects(student_id)

    # Step 4: Fetch content similarity scores and average them, filtering by subject
    content_scores = []
    student_interacted_programs = df_cleaned_filtered[df_cleaned_filtered['student_uuid'] == student_id]['program_uuid'].tolist()

    for program_id in svd_top_programs:
        if program_id in content_similarity_df.index:
            # Check if the program belongs to one of the student's top subjects
            program_subject = df_cleaned_filtered[df_cleaned_filtered['program_uuid'] == program_id]['subject'].values[0]
            if program_subject in student_top_subjects:
                # Extract the similarity values for this program
                similar_programs = content_similarity_df.loc[program_id]

                # Filter out already interacted programs directly, no sorting
                top_similar_programs = [p for p in similar_programs.index if p not in student_interacted_programs]

                if top_similar_programs:
                    # Ensure we're only calculating mean over valid scalar similarity scores
                    similarity_scores = [similar_programs[p] for p in top_similar_programs[:top_n] if np.isscalar(similarity_scores)]
                    
                    if similarity_scores:  # Only calculate mean if we have valid scores
                        avg_similarity_score = np.mean(similarity_scores)
                        
                        # Include only programs with a similarity score lower than 0.99
                        if avg_similarity_score < 0.99:
                            content_scores.append((program_id, avg_similarity_score))

    # Step 5: Normalize scores for SVD and content similarity
    max_svd_score = max([pred.est for pred in svd_predictions_sorted])
    max_content_score = max([score for pid, score in content_scores]) if content_scores else 1
    
    # Step 6: Combine the SVD scores with content-based similarity scores
    combined_recommendations = []
    for svd_program, svd_pred in zip(svd_top_programs[:top_n], svd_predictions_sorted[:top_n]):
        content_score = next((score for pid, score in content_scores if pid == svd_program), 0)  # Find corresponding content score
        
        # Normalize scores
        normalized_svd_score = svd_pred.est / max_svd_score if max_svd_score else svd_pred.est
        normalized_content_score = content_score / max_content_score if max_content_score else content_score
        
        combined_score = normalized_svd_score * 0.5 + normalized_content_score * 0.5  # Adjust the weight
        combined_recommendations.append((svd_program, combined_score))

    # Step 7: Sort the final recommendations based on the combined score and return the top N
    combined_recommendations.sort(key=lambda x: x[1], reverse=True)
    return combined_recommendations[:top_n]


In [ ]:
# Print out the results
print(f"Top 5 recommended programs for student {student_id}:")
for program_id, score in hybrid_recommendations:
    program_name = df_cleaned_filtered.loc[df_cleaned_filtered['program_uuid'] == program_id, 'program_name'].values[0]
    slug = df_cleaned_filtered.loc[df_cleaned_filtered['program_uuid'] == program_id, 'slug'].values[0]
    print(f"Program UUID: {program_id}, Program Name: {program_name}, Slug: {slug}, Combined Score: {score}")


In [ ]:
# Save the recommendations DataFrame to a CSV file
#df_recommendations.to_csv('/kaggle/working/student_program_recommendations.csv', index=False)

# Output will be saved as a CSV file in the Kaggle environment and can be downloaded